In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
%config Completer.use_jedi = False
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
data.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
608,609,70,RL,78.0,12168,Pave,NaN,Reg,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,1Fam,2Story,8,6,1934,1998,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,PConc,Gd,TA,Mn,BLQ,428,Unf,0,537,965,GasA,TA,Y,SBrkr,1940,1254,0,3194,0,0,2,1,4,1,TA,10,Typ,2,Gd,Basment,1934.0,Unf,2,380,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,9,2007,WD,Alloca,359100
1055,1056,20,RL,104.0,11361,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,5,1976,1976,Gable,CompShg,Plywood,Plywood,BrkFace,160.0,TA,TA,CBlock,Gd,TA,No,ALQ,644,Unf,0,549,1193,GasA,TA,Y,SBrkr,1523,0,0,1523,0,1,2,0,3,1,TA,7,Typ,1,TA,Attchd,1976.0,Fin,2,478,TA,TA,Y,0,0,0,0,189,0,NaN,MnPrv,NaN,0,5,2008,COD,Abnorml,180000
1356,1357,20,RL,NaN,9477,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1966,1966,Gable,CompShg,HdBoard,HdBoard,BrkFace,65.0,TA,TA,CBlock,TA,TA,No,Rec,340,Unf,0,524,864,GasA,TA,Y,SBrkr,892,0,0,892,0,0,1,0,3,1,TA,5,Typ,0,NaN,Attchd,1966.0,RFn,1,264,TA,TA,Y,0,0,0,0,0,0,NaN,GdWo,NaN,0,10,2008,WD,Normal,110000
1289,1290,60,RL,86.0,11065,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,Stone,788.0,Gd,TA,PConc,Gd,TA,Mn,Unf,0,Unf,0,1085,1085,GasA,Ex,Y,SBrkr,1120,850,0,1970,0,0,2,1,3,1,Ex,8,Typ,1,Gd,BuiltIn,2006.0,Fin,3,753,TA,TA,Y,177,74,0,0,0,0,NaN,NaN,NaN,0,10,2006,New,Partial,281000
580,581,20,RL,NaN,14585,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1960,1987,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,85.0,TA,TA,CBlock,TA,TA,No,BLQ,594,Rec,219,331,1144,GasA,Ex,Y,SBrkr,1429,0,0,1429,0,1,1,0,3,1,Gd,7,Typ,2,Gd,Attchd,1960.0,Unf,2,572,TA,TA,Y,216,110,0,0,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal,181900


In [3]:
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

### Data Exploration



In [4]:
# Let's see what columns we have in the dataset
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
# Let's find summary of numerical column
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
# # We will plo histogram to check how disperse our data is
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# for column_name  in tqdm(data.columns):
#     if data[column_name].dtypes == "int64":
#         data[column_name].hist(bins = 50, figsize = (5,5))
#         plt.xlabel(column_name)
#         plt.show()

As we can see fro the histogram above , some of the feature are skewed, and some are sparse.
So We nweed to transorm such kind of data , Because it is hard for a machine learning model to find pattern.

In [8]:
# from sklearn.model_selection import train_test_split
# import numpy as np
# def train_test_split_custom(data, test_ratio):
#     '''
#     input to function - 
#         data : Your dataset
#         test_ration: percentage of data needed as a test set
    
#     Output of Function -:
#         train data, test_data
#     '''
    
#     np.random.seed(42)
#     shuffled_indices = np.random.permutation(len(data))
#     test_size = int(len(data) * test_ratio)
#     test_data_index = shuffled_indices[:test_size]
#     train_data_index = shuffled_indices[test_size:]
    
#     return data.iloc[train_data_index],  data.iloc[test_data_index]

In [9]:
# train, test = train_test_split_custom(data, 0.1)

# print(" shape of train is {}".format(train.shape))

# print(" shape of train is {}".format(test.shape))

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

def func_na(x,temp):
    if pd.isna(x):
        return np.random.choice(temp)
    else:
         return x

def data_preprocessing(data, flag):
    numerical_column = [column for column in data.columns if data[column].dtypes == "int64"  or data[column].dtypes == "float64"]

    categorical_column = [column for column in data.columns if data[column].dtypes == "object"]

    num_df = data[numerical_column]
    cat_df = data[categorical_column]

    if flag == 'train':
        sales_price_label = num_df['SalePrice']

    # I want to Remove column year YearBuilt , YearRemodAdd , YrSold. I will add column stating how many years old house is

        num_df['YearsOld'] = num_df.apply(lambda x : x['YrSold'] - x['YearBuilt'], axis = 1)
        num_df.drop(['YrSold', 'YearBuilt', 'Id' , 'SalePrice'], axis = 1, inplace = True)
    else:
        num_df['YearsOld'] = num_df.apply(lambda x : x['YrSold'] - x['YearBuilt'], axis = 1)
        num_df.drop(['YrSold', 'YearBuilt', 'Id' ], axis = 1, inplace = True)

    OneHotEncoding_columns = ['LotShape','MSZoning', 'Street', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope','Neighborhood','Condition1', 'Condition2','BldgType',
                             'HouseStyle', 'RoofMatl','Exterior1st', 'Exterior2nd' ,'Foundation','Heating', 'CentralAir','Functional','Electrical','MasVnrType',
                             'GarageType', 'GarageFinish','PavedDrive','SaleType','SaleCondition','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
    label_encoding = [ 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond','HeatingQC','KitchenQual',
                     'FireplaceQu', 'GarageQual','GarageCond' ]

    replace = { 'Po':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4}
    delete_column = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']

#     fill_previous_value = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','GarageType', 'GarageFinish','GarageQual',
#                            'GarageCond','FireplaceQu']
#     fill_previous_value = [clm for clm in data.columns if data[clm].isna().sum()>0 ]
    fill_previous_value = label_encoding + OneHotEncoding_columns

    cat_df.drop(delete_column, inplace =True, axis = 1)

    for nan_column in fill_previous_value:
        temp = cat_df[nan_column].unique().copy()
        temp = temp[~pd.isna(temp)]

        cat_df[nan_column] = cat_df.apply(lambda x : func_na(x[nan_column], temp), axis = 1)

    # Now le
    one_hot_df = cat_df[OneHotEncoding_columns]
    label_encod_df = cat_df[label_encoding]


    encod = OneHotEncoder()

    encod.fit(one_hot_df)
    one_hot_trans_df = encod.transform(one_hot_df)
    feature_name = encod.get_feature_names(OneHotEncoding_columns)
    label_encod_df.replace(replace, inplace=True)
    

    num_pipeline = Pipeline([
     ('imputer', SimpleImputer(strategy="median")),
     ('std_scaler', StandardScaler()),
     ])
    housing_num_tr = num_pipeline.fit_transform(num_df)

    frame = [pd.DataFrame(one_hot_trans_df.toarray(), columns = feature_name), label_encod_df , pd.DataFrame(housing_num_tr, columns = num_df.columns)]
    
    print(pd.DataFrame(one_hot_trans_df.toarray(), columns = feature_name).shape, label_encod_df.shape, pd.DataFrame(housing_num_tr).shape)
    result = pd.concat(frame, axis = 1)

    if flag == 'train':
        return result, sales_price_label, encod
    else:
        return result, encod

In [11]:
result_train, sales_price_label , encod= data_preprocessing(data, 'train')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

(1460, 192) (1460, 9) (1460, 35)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [12]:
numerical_column = [column for column in test.columns if test[column].dtypes == "int64"  or test[column].dtypes == "float64"]

categorical_column = [column for column in test.columns if test[column].dtypes == "object"]

num_df = test[numerical_column]
cat_df = test[categorical_column]

# if flag == 'train':
#     sales_price_label = num_df['SalePrice']

# I want to Remove column year YearBuilt , YearRemodAdd , YrSold. I will add column stating how many years old house is

num_df['YearsOld'] = num_df.apply(lambda x : x['YrSold'] - x['YearBuilt'], axis = 1)
num_df.drop(['YrSold', 'YearBuilt', 'Id'], axis = 1, inplace = True)
# else:
#     num_df['YearsOld'] = num_df.apply(lambda x : x['YrSold'] - x['YearBuilt'], axis = 1)
#     num_df.drop(['YrSold', 'YearBuilt', 'Id' ], axis = 1, inplace = True)

OneHotEncoding_columns = ['LotShape','MSZoning', 'Street', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope','Neighborhood','Condition1', 'Condition2','BldgType',
                         'HouseStyle', 'RoofMatl','Exterior1st', 'Exterior2nd' ,'Foundation','Heating', 'CentralAir','Functional','Electrical','MasVnrType',
                         'GarageType', 'GarageFinish','PavedDrive','SaleType','SaleCondition','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
label_encoding = [ 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond','HeatingQC','KitchenQual',
                 'FireplaceQu', 'GarageQual','GarageCond' ]

replace = { 'Po':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4}
delete_column = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']

#     fill_previous_value = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','GarageType', 'GarageFinish','GarageQual',
#                            'GarageCond','FireplaceQu']
#     fill_previous_value = [clm for clm in data.columns if data[clm].isna().sum()>0 ]
fill_previous_value = label_encoding + OneHotEncoding_columns

cat_df.drop(delete_column, inplace =True, axis = 1)

for nan_column in fill_previous_value:
    temp = cat_df[nan_column].unique().copy()
    temp = temp[~pd.isna(temp)]

    cat_df[nan_column] = cat_df.apply(lambda x : func_na(x[nan_column], temp), axis = 1)

# Now le
one_hot_df = cat_df[OneHotEncoding_columns]
label_encod_df = cat_df[label_encoding]


# encod = OneHotEncoder()

# encod.fit(one_hot_df)
one_hot_trans_df = encod.transform(one_hot_df)
feature_name = encod.get_feature_names(OneHotEncoding_columns)
label_encod_df.replace(replace, inplace=True)


num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('std_scaler', StandardScaler()),
 ])
housing_num_tr = num_pipeline.fit_transform(num_df)

frame = [pd.DataFrame(one_hot_trans_df.toarray(), columns = feature_name), label_encod_df , pd.DataFrame(housing_num_tr, columns = num_df.columns)]

print(pd.DataFrame(one_hot_trans_df.toarray(), columns = feature_name).shape, label_encod_df.shape, pd.DataFrame(housing_num_tr).shape)
result = pd.concat(frame, axis = 1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

(1459, 192) (1459, 9) (1459, 35)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [13]:
result.shape, result_train.shape

((1459, 236), (1460, 236))

In [14]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(result_train, sales_price_label)
pred = lin_reg.predict(result)

In [15]:
from sklearn.tree import DecisionTreeRegressor
lin_reg = DecisionTreeRegressor()
lin_reg.fit(result_train, sales_price_label)
pred = lin_reg.predict(result)

In [16]:
sub = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
sub.head(2)

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989


In [17]:
subm = pd.DataFrame()

subm['Id'] = list(test['Id'])
subm['SalePrice'] = pred.tolist()

In [18]:
sub.to_csv('/kaggle/working/Submission.csv', index = False)